In [10]:
!pip install llama-index openai
!pip install -q --progress-bar off llama-index-readers-docling \
llama-index-node-parser-docling llama-index-embeddings-huggingface \
llama-index-llms-huggingface-api llama-index-vector-stores-milvus \
llama-index-readers-file
!pip install  --upgrade openai
1+1


2

In [8]:
!pip install  llama_index.readers.docling


In [1]:
from llama_index.readers.docling import DoclingReader

/opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from pathlib import Path
from tempfile import mkdtemp
from warnings import filterwarnings
from dotenv import load_dotenv
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.llms.openai import OpenAI
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.readers.docling import DoclingReader
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core.chat_engine import CondenseQuestionChatEngine
from llama_index.core.memory import ChatMemoryBuffer


load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:

def _get_env_from_colab_or_os(key):
    try:
        from google.colab import userdata

        try:
            return userdata.get(key)
        except userdata.SecretNotFoundError:
            pass
    except ImportError:
        pass
    return os.getenv(key)


GEN_MODEL = HuggingFaceInferenceAPI(
    token=os.getenv("HF_TOKEN"),
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
)

In [3]:
#""
'''BAAI/bge-small-en-v1.5 - Beijing Academy of Artificial Intelligence (BAAI)
Sentence Embedding Model / Text Embedding Model
Specifically designed for English text.
Based on E5 architecture, which itself is a modification of the MiniLM 
(or similar lightweight Transformer) architecture optimized for embedding tasks.
~60 million parameters'''

EMBED_MODEL = HuggingFaceEmbedding("BAAI/bge-small-en-v1.5")
embed_dim = len(EMBED_MODEL.get_text_embedding("Burger"))#
print(embed_dim)

384


In [4]:
SOURCE = 'Internal.pdf'
#SOURCE = "https://arxiv.org/pdf/2408.09869"

In [5]:
reader = DoclingReader()
node_parser = MarkdownNodeParser()
MILVUS_URI = str(Path(mkdtemp())/ 'docling_ahtsham.db')
vector_store = MilvusVectorStore(uri=MILVUS_URI,dim=embed_dim,overwrite=True)
index = VectorStoreIndex.from_documents(
    documents=reader.load_data(SOURCE),
    transformations=[node_parser],
    storage_context=StorageContext.from_defaults(vector_store=vector_store),
    embed_model=EMBED_MODEL,
)


/opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
2025-07-07 17:35:42,418 [DEBUG][_create_connection]: Created new connection using: 818134216e054ff38453cec9a38aee5e (async_milvus_client.py:599)


In [6]:
models = ['gpt-3.5-turbo','text-davinci-003']

llm = OpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=512
)



In [19]:


#QUERY =  'How do you ensure software compliance (licensing) in an organization '
QUERY = "JSON-formatted   example   of   a   network   device   configuration from the provided document"
result = index.as_query_engine(similarity_top_k=10,llm=llm).query(QUERY)
print(f"Q: {QUERY}\nA: {result.response.strip()}\n\nSources:")
display([(n.text, n.metadata) for n in result.source_nodes])

Q: JSON-formatted   example   of   a   network   device   configuration from the provided document
A: ```
{ "Device": "Switch1", "Hostname": "sw1-corp", "Interface": "GigabitEthernet0/1", "Description": "Uplink to Router", "VLAN": 10, "IP_Address": "192.168.10.2", "Subnet_Mask": "255.255.255.0", "Admin_State": "up" }
```

Sources:


[('## Internal IT Administration: Questions and Answers\n\nThis document provides a comprehensive list of internal IT administration questions across software, hardware, networking, and VPN domains, with detailed answers. The content is structured for easy indexing and retrieval, and includes various formats (paragraphs, lists, tables, JSON) to challenge a RAG pipeline.',
  {'header_path': '/'}),
 ("## Networking\n\n- Question: How does DHCP work and what is an IP address conflict? 1.\n\nAnswer: DHCP (Dynamic   Host   Configuration   Protocol)   automates   IP   assignment.   A   client broadcasts a DHCP Discover, the server offers an IP (Offer), the client requests it, and the server acknowledges it with a lease. The client then configures that IP. An IP address conflict occurs when two devices have the same IP (for example, if one is statically set to an IP that DHCP assigns to another). This causes connectivity issues (packets go to the wrong device or neither). Conflicts are resolv

In [ ]:
from llama_index.query_engine.transform_query_engine import HyDEQueryTransform
query_engine = HyDEQueryTransform().as_query_engine(query_engine)

In [20]:
result.source_nodes

[NodeWithScore(node=TextNode(id_='d3eb68b3-e39f-4a8d-8206-fe7e6011f3b9', embedding=None, metadata={'header_path': '/'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='04855bf1-9cbd-4d22-a3e8-0523857d0fb4', node_type='4', metadata={}, hash='9f213321a00b85bf4435f6adc3dabeae885b47083972dd8ad7c7500b1e3c511a'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='1c4624af-c778-42c9-a488-ab1636a5c71f', node_type='1', metadata={'header_path': '/'}, hash='257a0897aa55c114fee09468378d025c6963b9fe88f25e6c77f82833dec64e67')}, metadata_template='{key}: {value}', metadata_separator='\n', text='## Internal IT Administration: Questions and Answers\n\nThis document provides a comprehensive list of internal IT administration questions across software, hardware, networking, and VPN domains, with detailed answers. The content is structured for easy indexing and retrieval, and includes various formats (paragraphs, l

In [15]:
from llama_index.core.prompts import PromptTemplate
from llama_index.core.query_engine import RetrieverQueryEngine

CITATION_QA_TEMPLATE = PromptTemplate(
    "You are an expert IT assistant. Use the context below to answer the user's question. "
    "Please provide an answer based solely on the provided sources. "
    "Do not create examples or code on your own, Only retreive from the document if available"
    "When referencing information from a source, "
    "If the answer in the context is written in bullet points or numbered lists, preserve that format in your answer. "
    ""
    "When referencing information from a source, "
    "cite the appropriate source(s) using their corresponding numbers. "
    "Every answer should include at least one source citation. "
    "Only cite a source when you are explicitly referencing it. "
    "If none of the sources are helpful, you should indicate that. "
    "For example:\n"
    "Source 1:\n"
    "The sky is red in the evening and blue in the morning.\n"
    "Source 2:\n"
    "Water is wet when the sky is red.\n"
    "Query: When is water wet?\n"
    "Answer: Water will be wet when the sky is red [2], "
    "which occurs in the evening [1].\n"
    "Now it's your turn. Below are several numbered sources of information:"
    "\n------\n"
    "Context:\n{context_str}\n\n"
    "Question: {query_str}\n"
    "Answer:"
)


CITATION_REFINE_TEMPLATE = PromptTemplate(
    "You are an expert IT assistant. Use the context below to answer the user's question. "
    "Please provide an answer based solely on the provided sources. "
    "Do not create examples or code on your own, Only retreive from the document if available"
    "When referencing information from a source, "
    "If the answer in the context is written in bullet points or numbered lists, preserve that format in your answer. "
    "If the answer is a code block then return the code block"
    "When referencing information from a source, "
    "cite the appropriate source(s) using their corresponding numbers. "
    "Every answer should include at least one source citation. "
    "Only cite a source when you are explicitly referencing it. "
    "If none of the sources are helpful, you should indicate that. "
    "For example:\n"
    "Source 1:\n"
    "The sky is red in the evening and blue in the morning.\n"
    "Source 2:\n"
    "Water is wet when the sky is red.\n"
    "Query: When is water wet?\n"
    "Answer: Water will be wet when the sky is red [2], "
    "which occurs in the evening [1].\n"
    "Now it's your turn. Below are several numbered sources of information:"
    "\n------\n"
    "Context:\n{context_str}\n\n"
    "Question: {query_str}\n"
    "Answer:"
)

query_engine = RetrieverQueryEngine.from_args(
    retriever=index.as_retriever(similarity_top_k=5),
    text_qa_template=CITATION_QA_TEMPLATE,
    refine_template=CITATION_REFINE_TEMPLATE,
    llm=llm,
)

In [17]:
response = query_engine.query(" Provide a JSON-formatted example of a network device configuration for documentation")
print(response)

```
{ "Device": "Switch1", "Hostname": "sw1-corp", "Interface": "GigabitEthernet0/1", "Description": "Uplink to Router", "VLAN": 10, "IP_Address": "192.168.10.2", "Subnet_Mask": "255.255.255.0", "Admin_State": "up" }
```


In [ ]:
!pip install llama_index.query_engine.transform_query_engine

In [21]:
from llama_index.query_engine.transform_query_engine import HyDEQueryTransform
query_engine = HyDEQueryTransform().as_query_engine(query_engine,
                                                    retriever=index.as_retriever(similarity_top_k=5),
    text_qa_template=CITATION_QA_TEMPLATE,
    refine_template=CITATION_REFINE_TEMPLATE,
    llm=llm)

response = query_engine.query(" Provide a JSON-formatted example of a network device configuration for documentation")
print(response)

ModuleNotFoundError: No module named 'llama_index.query_engine'